In [1]:
pip install selenium bs4

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python312\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [71]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [139]:
options=Options()
options.add_argument('--headless')
driver=webdriver.Chrome(options=options)


In [142]:
def get_results_input(country_name="",course=""):
    url="https://www.timeshighereducation.com/world-university-rankings/latest/world-ranking"
    driver.get(url)

    if(country_name!=""):
        country=driver.find_element(By.XPATH,'//*[@id="location_chosen"]/ul/li/input')
        country.click()
        country.send_keys(country_name)
        country.send_keys(Keys.ENTER)

    if (course!=""):
        course_dropdown = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="subjects_chosen"]/a'))
        )
        course_dropdown.click()  
        
        course_input = driver.find_element(By.XPATH, '//*[@id="subjects_chosen"]/div/div/input')
        course_input.send_keys(course)  
        course_input.send_keys(Keys.ENTER) 

    wait = WebDriverWait(driver, 10)
    university_list=[]

    # Getting Scores
    table = wait.until(EC.presence_of_element_located((By.ID, 'datatable-1')))
    
    headers = []
    header_row = table.find_element(By.TAG_NAME, "thead").find_elements(By.TAG_NAME, "th")
    for header in header_row:
        headers.append(header.text.strip())
    
    table=driver.find_element(By.ID,'datatable-1')
    
    rows=table.find_elements(By.TAG_NAME,"tr")
    
    for row in rows:
        cells=row.find_elements(By.TAG_NAME,"td")
        dicty={}
        for i,cell in enumerate(cells):
            if i==1:
                name_country = cell.find_element(By.TAG_NAME, "a")
                name = name_country.text.strip()
                link = name_country.get_attribute("href")  
                name1=headers[i].split("\n")[0]
                name2=headers[i].split("\n")[1]
                dicty["link"]=link
                dicty[name1]=cell.text.split("\n")[0]
                dicty[name2]=cell.text.split("\n")[1]
            else:
                dicty[headers[i]]=cell.text
        university_list.append(dicty)
    
    #Getting Stats
    stats_button=driver.find_element(By.XPATH,'//*[@id="block-system-main"]/div/div[3]/div/div[1]/div[1]/div/div[1]/ul/li[2]/label/span')
    driver.execute_script("arguments[0].click();", stats_button)
    
    table = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.ID, 'datatable-1')) 
    )
    headers = []
    header_row = table.find_element(By.TAG_NAME, "thead").find_elements(By.TAG_NAME, "th")
    
    for header in header_row:
        headers.append(header.text.strip())
    
    rows=table.find_elements(By.TAG_NAME,"tr")
    
    for j,row in enumerate(rows):
        cells=row.find_elements(By.TAG_NAME,"td")
        for i,cell in enumerate(cells):
            if(i==0 or i==1):continue
            else:
                university_list[j][headers[i]]=cell.text

    return university_list[1:]
    
university_list = get_results_input()

In [143]:
university_list

[{'Rank': '1',
  'link': 'https://www.timeshighereducation.com/world-university-rankings/university-oxford',
  'Name': 'University of Oxford',
  'Country/Region': 'United Kingdom',
  'Overall': '98.5',
  'Teaching': '96.8',
  'Research Environment': '100.0',
  'Research Quality': '98.8',
  'Industry': '99.6',
  'International Outlook': '97.3',
  'No. of FTE Students': '22,095',
  'No. of students per staff': '10.8',
  'International Students': '43%',
  'Female:Male Ratio': '51 : 49'},
 {'Rank': '2',
  'link': 'https://www.timeshighereducation.com/world-university-rankings/massachusetts-institute-technology',
  'Name': 'Massachusetts Institute of Technology',
  'Country/Region': 'United States',
  'Overall': '98.1',
  'Teaching': '99.2',
  'Research Environment': '96.0',
  'Research Quality': '99.7',
  'Industry': '100.0',
  'International Outlook': '93.8',
  'No. of FTE Students': '11,836',
  'No. of students per staff': '8.0',
  'International Students': '33%',
  'Female:Male Ratio': 